In [11]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim, tensor
from classic_pipeline import *
from utilities import *
import numpy
import torch

In [2]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [3]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2), AtomicType.CONJUNCTION: Dim(2), AtomicType.PUNCTUATION: Dim(2), AtomicType.NOUN_PHRASE: Dim(2), AtomicType.PREPOSITIONAL_PHRASE: Dim(2)})
spider_ansatz = SpiderAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2)})
mps_ansatz = MPSAnsatz({AtomicType.NOUN: Dim(2), AtomicType.SENTENCE: Dim(2)}, bond_dim = 3)

In [ ]:
#data-extracion for classic pipeline

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

In [8]:
save_data("train_data.txt", train_labels, train_circuits)
save_data("test_data.txt", test_labels, test_circuits)

In [22]:
train_tensors = []
test_tensors = []

for circuit in train_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    train_tensors.append(tensor)

for circuit in test_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    test_tensors.append(tensor)

In [19]:
max_dim = max([len(circuit) for circuit in train_circuits])
padded_train_circuits = [tensor.pad() for diagram in train_circuits]

"""max_dim = max([len(circuit) for circuit in test_circuits])
padded_test_circuits = [diagram.pad(max_dim) for diagram in test_circuits]"""

276


'max_dim = max([len(circuit) for circuit in test_circuits])\npadded_test_circuits = [diagram.pad(max_dim) for diagram in test_circuits]'

In [ ]:
train_set, test_set = pip.create_dataset(train_labels, train_circuits), pip.create_dataset(test_labels, test_circuits)
pip.create_trainer(train_circuits, test_circuits)
pip.train_model(train_set, test_set)
pip.plot() 

In [ ]:
from lambeq import BobcatParser, AtomicType
from lambeq import TensorAnsatz
import discopy
# Define atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

sentence = 'John walks in the park'

# Get a string diagram
parser = BobcatParser(verbose='text')
diagram = parser.sentence2diagram(sentence)

tensor_ansatz = TensorAnsatz({N: discopy.Dim(2), S: discopy.Dim(2)})
tensor_diagram = tensor_ansatz(diagram)
new = tensor_diagram.to_tn()
print(new)